<a href="https://colab.research.google.com/github/Dwarkamoye/CSEE5590-CS490-0004-AI-for-Cybersecurity/blob/master/ICP-7/L4_Federated_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Section: Federated Learning

# Lesson: Introducing Federated Learning

Federated Learning is a technique for training Deep Learning models on data to which you do not have access. Basically:

Federated Learning: Instead of bringing all the data to one machine and training a model, we bring the model to the data, train it locally, and merely upload "model updates" to a central server.

Use Cases:

    - app company (Texting prediction app)
    - predictive maintenance (automobiles / industrial engines)
    - wearable medical devices
    - ad blockers / autotomplete in browsers (Firefox/Brave)
    
Challenge Description: data is distributed amongst sources but we cannot aggregated it because of:

    - privacy concerns: legal, user discomfort, competitive dynamics
    - engineering: the bandwidth/storage requirements of aggregating the larger dataset

# Lesson: Introducing / Installing PySyft

In order to perform Federated Learning, we need to be able to use Deep Learning techniques on remote machines. This will require a new set of tools. Specifically, we will use an extensin of PyTorch called PySyft.

### Install PySyft

The easiest way to install the required libraries is with [Conda](https://docs.conda.io/projects/conda/en/latest/user-guide/overview.html). Create a new environment, then install the dependencies in that environment. In your terminal:

```bash
conda create -n pysyft python=3
conda activate pysyft # some older version of conda require "source activate pysyft" instead.
conda install jupyter notebook
pip install syft
pip install numpy
```

If you have any errors relating to zstd - run the following (if everything above installed fine then skip this step):

```
pip install --upgrade --force-reinstall zstd
```

and then retry installing syft (pip install syft).

If you are using Windows, I suggest installing [Anaconda and using the Anaconda Prompt](https://docs.anaconda.com/anaconda/user-guide/getting-started/) to work from the command line. 

With this environment activated and in the repo directory, launch Jupyter Notebook:

```bash
jupyter notebook
```

and re-open this notebook on the new Jupyter server.



In [1]:
!pip install syft
!pip install numpy

In [0]:
import torch as th

In [3]:
x = th.tensor([1,2,3,4,5])
x

tensor([1, 2, 3, 4, 5])

In [0]:
y = x + x

In [5]:
print(y)

tensor([ 2,  4,  6,  8, 10])


In [6]:
import syft as sy

W0702 23:37:32.062291 139955401922432 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0702 23:37:32.082748 139955401922432 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [0]:
hook = sy.TorchHook(th)  # this line of code create an instance of th with its backend APIs modified with Syft functions

In [8]:
th.tensor([1,2,3,4,5]) # notice that Torch functionalities still behave the same

tensor([1, 2, 3, 4, 5])

# Lesson: Basic Remote Execution in PySyft

## PySyft => Remote PyTorch

The essence of Federated Learning is the ability to train models in parallel on a wide number of machines. Thus, we need the ability to tell remote machines to execute the operations required for Deep Learning.

Thus, instead of using Torch tensors - we're now going to work with **pointers** to tensors. Let me show you what I mean. First, let's create a "pretend" machine owned by a "pretend" person - we'll call him Bob.

In [0]:
bob = sy.VirtualWorker(hook, id="bob") # creates a virtual worker (a simulaion to interface to Bob machines)

In [10]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5])

In [0]:
x_ptr =  x.send(bob) # send this data to bob

In [13]:
bob._objects

{78027814325: tensor([1, 2, 3, 4, 5])}

In [14]:
x_ptr

(Wrapper)>[PointerTensor | me:56910295265 -> bob:78027814325]

In [16]:
x_ptr.location

<VirtualWorker id:bob #objects:1>

In [18]:
x_ptr.id_at_location

78027814325

In [19]:
x.id

78027814325

In [20]:
x.owner

<VirtualWorker id:me #objects:0>

In [21]:
hook.local_worker

<VirtualWorker id:me #objects:0>

In [24]:
x = x_ptr.get()
x

tensor([1, 2, 3, 4, 5])

In [25]:
bob._objects

{}

# Project: Experience with Remote Tensors

In this project, I want you to .send() and .get() a tensor to TWO workers by calling .send(bob,alice). This will first require the creation of another VirtualWorker called alice.

In [0]:
# create a second Virtual worker and call it Alice


In [0]:
# 1 - create some data (a tensor)
# 2- send the data to bob and alice


In [0]:
# notice what does the send fucntion on two workers return; a multi-pointer
x_ptr

In [0]:
# what does .child on the pointer object return?
# what does .child.child on the pointer object return? 

In [0]:
# try the .get() on the pointer


In [0]:
# try the .get(sum_results=True) on your pointer


# Lesson: Introducing Remote Arithmetic

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)

In [27]:
x # a pointer to a remote tensor located at bob

(Wrapper)>[PointerTensor | me:70302368637 -> bob:46217313090]

In [28]:
y # a pointer to another remote tensor located at bob

(Wrapper)>[PointerTensor | me:43386618481 -> bob:43057853571]

In [29]:
z = x + y # treat those tensors (i.e., x and y) as local tensors, but they are actually executed remotely
z

(Wrapper)>[PointerTensor | me:54205963104 -> bob:2528350893]

In [30]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [31]:
z = th.add(x,y) # we can also use Torch operation on these tensors to be executed remotely
z

(Wrapper)>[PointerTensor | me:21939976965 -> bob:99334304901]

In [32]:
z = z.get() # yup! same results!
z

tensor([2, 3, 4, 5, 6])

In [0]:
x = th.tensor([1.,2,3,4,5], requires_grad=True).send(bob)
y = th.tensor([1.,1,1,1,1], requires_grad=True).send(bob)

In [34]:
z = (x + y).sum()
z = z.get()
z

tensor(20., requires_grad=True)

In [0]:
z.backward() 

In [0]:
x = x.get()

In [37]:
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [0]:
x.grad

# Project: Learn a Simple Linear Model

In this project, I'd like for you to create a simple linear model which will solve for the following dataset below. You should use only Variables and .backward() to do so (no optimizers or nn.Modules). 

Furthermore, you must do so with both the data and the model being located on Bob's machine.

In [39]:
import torch as th
import syft as sy

hook = sy.TorchHook(th)  
bob = sy.VirtualWorker(hook, id="bob")

bob

W0702 23:39:55.895015 139955401922432 hook.py:98] Torch was already hooked... skipping hooking process


<VirtualWorker id:bob #objects:3>

In [0]:
# create some toy data for our model
input_data = th.tensor([[1.,1],[0.5,1],[1,0],[0,0]], requires_grad=True).send(bob)
output_data = th.tensor([[1.],[1],[0],[0]], requires_grad=True).send(bob)

In [0]:
# send the data to bob
x_ptr = input_data.send(bob)

In [0]:
# create some linear weights and send them to bob
weights = th.tensor([[0.01],[0.01]], requires_grad = True).send(bob)

In [48]:
# create a linear model and train it on Bob's machine
# remember how to create a linear model? No? :( 
# Here's the main steps:
#    1- find a prediction
#    2- calcualte the loss (a mean square loss)
#    3- backpropogate using the backword() function
#    4- DO NOT forget to clear your gradients after updating the weights
#    weights.data.sub_(weights.grad * lr)

prediction = input_data.mm(weights)
prediction

PureTorchTensorFoundError: ignored

In [49]:
lr = 0.1

for i in range(10):
  prediction = input_data.mm(weights)
  loss = ((prediction - output_data)**2).mean()
  loss.backward()
  weights.data.sub_(weights.grad * lr)
  weights.grad *= 0

  print(loss.get().data)

PureTorchTensorFoundError: ignored

# Lesson: Garbage Collection and Common Errors


In [0]:
bob = bob.clear_objects() # clear the contents of a remote object

In [51]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [53]:
bob._objects

{}

In [0]:
del x  # delete the pointer to the remote object

In [55]:
bob._objects

{}

In [56]:
x.child.garbage_collect_data  # True by default

NameError: ignored

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [58]:
bob._objects

{}

In [0]:
x = "asdf"

In [60]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [62]:
x

(Wrapper)>[PointerTensor | me:23345322892 -> bob:87941201255]

In [63]:
bob._objects

{}

In [0]:
x = "asdf"

In [65]:
bob._objects  # some error from Jupyter is casuing this to appear!

{}

In [0]:
del x

In [67]:
bob._objects

{}

In [68]:
bob = bob.clear_objects() # erase force 
bob._objects

{}

In [0]:

for i in range(1000):
    x = th.tensor([1,2,3,4,5]).send(bob)

In [70]:
bob._objects # notice that there is only a single tensor in bob

{}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1])

In [72]:
z = x + y

TensorsNotCollocatedException: ignored

In [0]:
alice = sy.VirtualWorker(hook, id="alice")
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(alice)

In [74]:
z = x + y

TensorsNotCollocatedException: ignored

# Lesson: Toy Federated Learning

Let's start by training a toy model the centralized way. This is about a simple as models get. We first need:

- a toy dataset
- a model
- some basic training logic for training a model to fit the data.

In [0]:
from torch import nn, optim

In [0]:
# A Toy Dataset
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [0]:
# A Toy Model
model = nn.Linear(2,1)

In [0]:
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [79]:
def train(iterations=10):
    for iter in range(iterations):
        opt.zero_grad()

        pred = model(data)

        loss = ((pred - target)**2).sum()

        loss.backward()

        opt.step()

        print(loss.data)
        
train()

tensor(2.1441)
tensor(0.6496)
tensor(0.3639)
tensor(0.2309)
tensor(0.1491)
tensor(0.0967)
tensor(0.0629)
tensor(0.0410)
tensor(0.0268)
tensor(0.0176)


In [80]:
# nothing was federated up to this point!

import torch as th
import syft as sy

hook = sy.TorchHook(th)  
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")



W0702 23:44:13.044775 139955401922432 hook.py:98] Torch was already hooked... skipping hooking process


In [0]:
# create local datasets at Bob and Alice
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

data_alice = data[2:4].send(alice)
target_alice = target[2:4].send(alice)

In [82]:
datasets = [(data_bob, target_bob), (data_alice, target_alice)]
datasets

[((Wrapper)>[PointerTensor | me:39258891273 -> bob:27223911510],
  (Wrapper)>[PointerTensor | me:78463271086 -> bob:43466391012]),
 ((Wrapper)>[PointerTensor | me:42126628544 -> alice:27651519238],
  (Wrapper)>[PointerTensor | me:53598640678 -> alice:31201241879])]

In [0]:
def train(iterations=1):

    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    
    for iter in range(iterations):

        for _data, _target in datasets:

            # send model to the data
            model = model.send(_data.location)

            # do training on the remote machine
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()

            # get smarter model back
            model = model.get()

            print(loss.get())

In [84]:
train()

tensor(6.6849, requires_grad=True)
tensor(0.2845, requires_grad=True)


# Lesson: Advanced Remote Execution Tools

In the last section we trained a toy model using Federated Learning. We did this by calling .send() and .get() on our model, sending it to the location of training data, updating it, and then bringing it back. However, at the end of the example we realized that we needed to go a bit further to protect people privacy. Namely, we want to average the gradients BEFORE calling .get(). That way, we won't ever see anyone's exact gradient (thus better protecting their privacy!!!)

But, in order to do this, we need a few more pieces:

- use a pointer to send a Tensor directly to another worker

And in addition, while we're here, we're going to learn about a few more advanced tensor operations as well which will help us both with this example and a few in the future!

In [85]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [0]:
x = x.send(alice)

In [88]:
bob._objects

{}

In [89]:
alice._objects

{}

In [90]:
y = x + x

KeyError: ignored

In [91]:
y

(Wrapper)>[PointerTensor | me:57643980546 -> alice:80988846488]

In [92]:
bob._objects

{}

In [93]:
alice._objects

{}

In [0]:
jon = sy.VirtualWorker(hook, id="jon")

In [95]:
bob.clear_objects()
alice.clear_objects()



<VirtualWorker id:alice #objects:0>

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)
y = th.tensor([1,2,3,4,5]).send(bob).send(jon)

In [97]:
z = x + y

TensorsNotCollocatedException: ignored

In [98]:
x = x.get() # to get the data back
x

(Wrapper)>[PointerTensor | me:93030015619 -> bob:82315151878]

In [99]:
bob._objects

{}

In [100]:
alice._objects

{}

In [101]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [102]:
bob._objects

{}

# Lesson: Pointer Chain Operations

In [103]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [105]:
bob._objects

{}

In [106]:
alice._objects

{}

In [107]:
x.move(alice)

KeyError: ignored

In [108]:
x

(Wrapper)>[PointerTensor | me:65981281085 -> bob:42836755700]

In [109]:
bob._objects

{}

In [110]:
alice._objects

{}

In [111]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [115]:
bob._objects

{}

In [116]:
alice._objects

{}

In [117]:
x

(Wrapper)>[PointerTensor | me:24536183205 -> alice:97696501371]

In [118]:
x.remote_get() # inplace operation

KeyError: ignored

In [119]:
bob._objects

{}

In [120]:
alice._objects

{}

In [121]:
x.move(bob)

KeyError: ignored

In [0]:
x

(Wrapper)>[PointerTensor | me:83688287202 -> bob:83688287202]

In [0]:
bob._objects

{83688287202: tensor([1, 2, 3, 4, 5])}

In [0]:
alice._objects

{}

**Exercise 1:**

- Create a toy dataset
- Create a toy linear model
- Train the dataset using Federated Learning, but do not share the gradiant with the main server, instead use the move function to send the gradients to one of the participating workers, aggregate the model, then send the final model to the main server (i.e., local worker).



---


**Exercise 2:**

While exercise 1 illustrates a good example of preseving privacy in FL, we still had to expose the gradients to one of the participating workers. To avoid this limitation, you need to send the gradiants to a TRUSTED thirdparty (trusted aggregator) who will aggregate the models and then send the final model to the server (local worker). In this way, we assure that none of the participating workers can access the aggregated model!

1. create a dataset for each worker (create two)
2. create a model for each worker and train it remotely on each worker
3. send those two models using the *move()* function to a third worker
4. the third workers aggregates the two models (find their mean)
5. send the aggregated model to the main server (local worker) using the *get()* function



---

To set the weights: `model.weight.set_()`



In [1]:
# Exercise 1
import syft as sy
import torch as th
from torch import nn, optim

W0703 02:01:27.642559 140388445550464 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0703 02:01:27.661717 140388445550464 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [0]:
hook = sy.TorchHook(th)  


In [0]:
# create workers
#bob = 
#alice = 
#secureWorker = 
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
secureWorker = sy.VirtualWorker(hook, id="secureWorker")

In [4]:
# Make each worker aware of the other workers
bob.add_workers([alice,secureWorker])
alice.add_workers([bob,secureWorker])
secureWorker.add_workers([bob, alice])

W0703 02:01:31.760077 140388445550464 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0703 02:01:31.764280 140388445550464 base.py:628] Worker secureWorker already exists. Replacing old worker which could cause                     unexpected behavior
W0703 02:01:31.766371 140388445550464 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior
W0703 02:01:31.767809 140388445550464 base.py:628] Worker secureWorker already exists. Replacing old worker which could cause                     unexpected behavior
W0703 02:01:31.770915 140388445550464 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior
W0703 02:01:31.775183 140388445550464 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior


<VirtualWorker id:secureWorker #objects:0>

In [0]:
# A Toy Dataset
#data = 
#target = 
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [6]:
# create local datasets at Bob and Alice
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

data_alice = data[2:4].send(alice)
target_alice = target[2:4].send(alice)

datasets = [(data_bob, target_bob), (data_alice, target_alice)]
datasets

[((Wrapper)>[PointerTensor | me:46397929170 -> bob:6111035636],
  (Wrapper)>[PointerTensor | me:64845251277 -> bob:47669843844]),
 ((Wrapper)>[PointerTensor | me:11640901858 -> alice:99994155504],
  (Wrapper)>[PointerTensor | me:95363125172 -> alice:76733740094])]

In [0]:
# create a linear model at local worker

model1 = nn.Linear(2,1)
model2 = nn.Linear(2,1)


In [0]:
# Send copies of the linear model to alice and bob

bob_model = model1.send(bob)
alice_model = model2.send(alice)

model_list =[bob_model,alice_model]

In [9]:
alice_model

Linear(in_features=2, out_features=1, bias=True)

In [10]:
bob_model

Linear(in_features=2, out_features=1, bias=True)

In [0]:
# train the models at alice and bob
# move the models to the third worker
# aggreegate the models
# send the model back to the local worker

def train(iterations=1):
    for i,val in enumerate(model_list):
              _data, _target = datasets[i]
              print("Dataset",_data)
              model = val
              opt = optim.SGD(params=model.parameters(), lr=0.1)
              # do training on the remote machine
              opt.zero_grad()
              pred = model(_data)
              loss = ((pred - _target)**2).sum()
              loss.backward()
              opt.step()

              # move to secure worker
              model.move(secureWorker)

              print(loss.get())
           

In [12]:
train()

Dataset (Wrapper)>[PointerTensor | me:46397929170 -> bob:6111035636]
tensor(3.0847, requires_grad=True)
Dataset (Wrapper)>[PointerTensor | me:11640901858 -> alice:99994155504]
tensor(0.5591, requires_grad=True)


In [16]:
secure_model = (bob_model.weight + alice_model.weight)/2
secure_model

(Wrapper)>[PointerTensor | me:92032297017 -> secureWorker:95661836232]